In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import folium
import datetime
import pickle
from shapely.geometry import Point

In [2]:
promise_zone = gpd.read_file('../data/MDHA_Promise_Zones/Export_Output_5.shp')
print(promise_zone.crs)

epsg:2274


In [3]:
promise_zone = promise_zone.to_crs(epsg = 4326)

In [4]:
# Import the file as a Pandas DataFrame
fp = '../data/july.csv'
july = pd.read_csv(fp)

In [5]:
company_dict = {'Bird':0, 'Lyft': 1, 'Gotcha': 2, 'Lime': 3, 'Spin': 4, 'Jump': 5, 'Bolt': 6}
july.companyname = july.companyname.replace(company_dict)

In [6]:
july.pubdatetime = pd.to_datetime(july['pubdatetime'])

In [7]:
july.to_pickle("../data/july.pkl")

In [8]:
july_scooters = july[['pubdatetime', 'latitude', 'longitude', 'sumdid', 'chargelevel', 'companyname']]

In [23]:
july_scooters['day_specific'] = july_scooters.pubdatetime.strftime('%d')

AttributeError: 'Series' object has no attribute 'strftime'

In [24]:
july_scooters['day'] = july_scooters['pubdatetime'].dt.day


C:\Users\RMAUR\AppData\Local\Temp/ipykernel_12936/2140293322.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  july_scooters['day'] = july_scooters['pubdatetime'].dt.day


In [26]:
july_scooters.tail()

,pubdatetime,latitude,longitude,sumdid,chargelevel,companyname,splice,day
25075440,2019-07-31 23:59:57,36.162420,-86.779314,Powered-1a8321d0-5c21-04a4-bd7d-7b96ba51f17b,100.00,6,0 2019-07-01 00:00:33.550\n1 ...,31
25075441,2019-07-31 23:59:57,36.146010,-86.810197,Powered-bd4ba83b-513f-ed5c-3c51-b4f51e7fdc51,100.00,6,0 2019-07-01 00:00:33.550\n1 ...,31
25075442,2019-07-31 23:59:57,36.165837,-86.777765,Powered-cc9c0b7e-2b9e-def9-e48b-1a8a489c4daf,71.68,6,0 2019-07-01 00:00:33.550\n1 ...,31
25075443,2019-07-31 23:59:57,36.149792,-86.806818,Powered-55a7ac02-7e17-bb3b-130d-b160d9776405,0.00,6,0 2019-07-01 00:00:33.550\n1 ...,31
25075444,2019-07-31 23:59:57,36.162644,-86.779345,Powered-03be23ca-d43b-222f-be54-e44b5b4690df,100.00,6,0 2019-07-01 00:00:33.550\n1 ...,31
